In [1]:
from numpy import *
from pathlib import Path

In [3]:
#FP-growth(Frequent Pattern,频繁模式)可以更有效地挖掘频繁项集，但不能用于发现关联规则：
##1）构建FP树。第一次遍历会获得每个元素项的出现频率，并去掉不满足最小支持度的元素；第二次遍历构建FP树
##2）从FP树中挖掘频繁项集

In [4]:
#创建PF树的数据结构

In [28]:
class treeNode:
    def __init__(self,nameValue,numOccur,parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}
        
    def inc(self,numOccur):
        self.count += numOccur
    def disp(self,ind=1):
        print('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

In [24]:
def createTree(dataSet,minSup=1):
    headerTable = {}            #头指针表
    for trans in dataSet:
        for item in trans:
            headerTable[item] = headerTable.get(item,0) + dataSet[trans]
            
    headerTableCopy = headerTable.copy()
    for k in headerTableCopy.keys():  #remove items not meeting minSup
        if headerTable[k] < minSup: 
            del(headerTable[k])
            
    freqItemSet = set(headerTable.keys())
    if len(freqItemSet) == 0:
        return None,None
    
    for k in headerTable:
        headerTable[k] = [headerTable[k],None]
        
    retTree = treeNode('Null Set',1,None)
    for tranSet,count in dataSet.items():
        localD = {}
        for item in tranSet:
            if item in freqItemSet:
                localD[item] = headerTable[item][0]                       #添加全局频率
                
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(),
                                                key=lambda p:p[1], reverse=True)]  #根据全局频率对元素进行排序
            updateTree(orderedItems,retTree,headerTable,count)
    return retTree,headerTable

In [15]:
def updateTree(items,inTree,headerTable,count):
    if items[0] in inTree.children:
        inTree.children[items[0]].inc(count)
    else:                                       #不存在时，则创建一个新的节点，作为一个子节点添加到树中
        inTree.children[items[0]] = treeNode(items[0],count,inTree)
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1],inTree.children[items[0]])
            
    if len(items) > 1:
        updateTree(items[1::], inTree.children[items[0]],headerTable,count)

def updateHeader(nodeToTest, targetNode):         #寻找空节点插入
    while nodeToTest.nodeLink != None:
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [16]:
##测试

In [17]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [18]:
simpDat = loadSimpDat()
simpDat

[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [19]:
initSet = createInitSet(simpDat)

In [20]:
initSet

{frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [31]:
myFPtree,myHeaderTab = createTree(initSet,3)

In [32]:
myFPtree.disp()

   Null Set   1
     z   5
       r   1
       x   3
         s   2
           t   2
             y   2
         y   1
           t   1
             r   1
     x   1
       s   1
         r   1


In [34]:
##从FP树挖掘频繁项集
###1)从FP树中获得条件模式基
###2)利用条件模式基，构建一个条件FP树
###3)迭代重复上述步骤，直到树包含一个元素项为止